# Proporcional Difference

Teste do método de seleção de features Proporcional Difference proposto em http://crpit.com/confpapers/CRPITV87Simeon.pdf

Este método foi testado contra outros, inclusive o SentiWordNet, em:
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.471.5694&rep=rep1&type=pdf#page=77

In [1]:
# Aqui estou usando todo o corpus para selecionar as melhores features.

from utils import  *
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import defaultdict
from pandas import DataFrame

# 1) Separar o corpus por classes
numRemover = NumRemover()
all_data = numRemover.fit_transform(get_data_from_db())
pos_data = numRemover.fit_transform(get_data_from_db(sentiment="PO"))["texts"]
neg_data = numRemover.fit_transform(get_data_from_db(sentiment="NG"))["texts"]
neu_data = numRemover.fit_transform(get_data_from_db(sentiment="NE"))["texts"]

# 2) Criar um vetorizer para unigrams
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode")

# 3) Fazer o fit para todo o corpus
vectorizer.fit(all_data["texts"])

# 4) Pegar todo o vocabulário do corpus
vocab = list(vectorizer.vocabulary_.keys())
vocab_indexed = {value: key for key, value in vectorizer.vocabulary_.items()}

# 5) Obter as matrizes das classes
pos_matrix = vectorizer.transform(pos_data)
neg_matrix = vectorizer.transform(neg_data)
neu_matrix = vectorizer.transform(neu_data)

# 6) Obter o vetor de soma das frquencias das palavras nas classes
pos_sum = pos_matrix.sum(axis=0)
neg_sum = neg_matrix.sum(axis=0)
neu_sum = neu_matrix.sum(axis=0)

# 7) Construir um dicionario de frequencias para cada classe e calcular o PD de cada uma
# função que calcula o PD
pd = lambda c1, c2, c3: max([(c1 - c2 - c3)/(c1 + c2 + c3), (c2 - c1 - c3)/(c1 + c2 + c3), (c3 - c2 - c1)/(c1 + c2 + c3)])
freq_dict = [ {"PO": pos_sum[0,index], "NG": neg_sum[0,index], "NE": neu_sum[0,index], "PD": pd(pos_sum[0,index], neg_sum[0,index], neu_sum[0,index])} for index in vocab_indexed.keys()]
freq_df = DataFrame(data = freq_dict, index= vocab_indexed.values())
freq_df = freq_df[["PO", "NG", "NE", "PD"]] # Reordenando as colunas do data frame

# 8) Selecionar as melhores features
threshold = 1
selected = freq_df[freq_df["PD"] == threshold].index.values
selected = list(selected)
print("Dimensionalidade: ", len(selected))

# 9) Avaliar o desempenho dos classificadores utilizando as features selecionadas
unigram_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode", vocabulary= selected)
# evaluate(all_data, unigram_vectorizer, 10)

Dimensionalidade:  23444


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

print("Simple decision Tree")
run_cross_validation2(all_data, unigram_vectorizer, DecisionTreeClassifier(criterion="gini"))

Simple decision Tree
Cross Validation:
Fold  0  - Acuracia:  0.518867924528
Fold  1  - Acuracia:  0.538461538462
Fold  2  - Acuracia:  0.586538461538
Fold  3  - Acuracia:  0.519230769231
Fold  4  - Acuracia:  0.557692307692
Fold  5  - Acuracia:  0.528846153846
Fold  6  - Acuracia:  0.528846153846
Fold  7  - Acuracia:  0.509615384615
Fold  8  - Acuracia:  0.548076923077
Fold  9  - Acuracia:  0.519230769231
Accuracia media:  0.535540638607
Desvio padrão:  0.0219748005477


In [6]:
print("Random Forest")
run_cross_validation2(all_data, unigram_vectorizer, RandomForestClassifier(criterion="gini", n_estimators=50))

Random Forest
Cross Validation:
Fold  0  - Acuracia:  0.396226415094
Fold  1  - Acuracia:  0.346153846154
Fold  2  - Acuracia:  0.461538461538
Fold  3  - Acuracia:  0.432692307692
Fold  4  - Acuracia:  0.490384615385
Fold  5  - Acuracia:  0.413461538462
Fold  6  - Acuracia:  0.394230769231
Fold  7  - Acuracia:  0.384615384615
Fold  8  - Acuracia:  0.413461538462
Fold  9  - Acuracia:  0.413461538462
Accuracia media:  0.414622641509
Desvio padrão:  0.0382285092259


In [57]:
# Aqui usamos somente o conjunto de treinamento para selecionar as melhores features com PD

from utils import  *
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import defaultdict
from pandas import DataFrame

from sklearn.model_selection import train_test_split
import random 

# 1) Separar o corpus por classes
numRemover = NumRemover()
all_data = numRemover.fit_transform(get_data_from_db())
seed = int(random.uniform(0, 100))
train_ratio = 0.7
train, test, ytrain, ytest = train_test_split(all_data, all_data["labels"], train_size = train_ratio, stratify = all_data["labels"], random_state = seed)

pos_data = train[train["labels"] == "PO"]["texts"]
neg_data = train[train["labels"] == "NG"]["texts"]
neu_data = train[train["labels"] == "NE"]["texts"]

# 2) Criar um vetorizer para unigrams
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode")

# 3) Fazer o fit para todo o corpus
vectorizer.fit(train["texts"])

# 4) Pegar todo o vocabulário do corpus
vocab = list(vectorizer.vocabulary_.keys())
vocab_indexed = {value: key for key, value in vectorizer.vocabulary_.items()}

# 5) Obter as matrizes das classes
pos_matrix = vectorizer.transform(pos_data)
neg_matrix = vectorizer.transform(neg_data)
neu_matrix = vectorizer.transform(neu_data)

# 6) Obter o vetor de soma das frquencias das palavras nas classes
pos_sum = pos_matrix.sum(axis=0)
neg_sum = neg_matrix.sum(axis=0)
neu_sum = neu_matrix.sum(axis=0)

# 7) Construir um dicionario de frequencias para cada classe e calcular o PD de cada uma
# função que calcula o PD
pd = lambda c1, c2, c3: max([(c1 - c2 - c3)/(c1 + c2 + c3), (c2 - c1 - c3)/(c1 + c2 + c3), (c3 - c2 - c1)/(c1 + c2 + c3)])
freq_dict = [ {"PO": pos_sum[0,index], "NG": neg_sum[0,index], "NE": neu_sum[0,index], "PD": pd(pos_sum[0,index], neg_sum[0,index], neu_sum[0,index])} for index in vocab_indexed.keys()]
freq_df = DataFrame(data = freq_dict, index= vocab_indexed.values())
freq_df = freq_df[["PO", "NG", "NE", "PD"]] # Reordenando as colunas do data frame

# 8) Selecionar as melhores features
threshold = 1
selected = freq_df[freq_df["PD"] >= threshold].index.values
selected = list(selected)
print("Dimensionalidade: ", len(selected))

# 9) Avaliar o desempenho dos classificadores utilizando as features selecionadas
unigram_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode", vocabulary= selected)
evaluate(all_data, unigram_vectorizer, 10)

Dimensionalidade:  17728
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.635359216255
Desvio padrão:  0.0341172871526

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.732220609579
Desvio padrão:  0.0476956566556

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.593196661829
Desvio padrão:  0.0314476741683


In [8]:
# Aqui usamos somente o conjunto de treinamento para selecionar as melhores features com PD

from utils import  *
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import defaultdict
from pandas import DataFrame

from sklearn.model_selection import train_test_split
import random 

# 1) Separar o corpus por classes
numRemover = NumRemover()
all_data = numRemover.fit_transform(get_data_from_db())
seed = int(random.uniform(0, 100))
train_ratio = 0.7
train, test, ytrain, ytest = train_test_split(all_data, all_data["labels"], train_size = train_ratio, stratify = all_data["labels"], random_state = seed)

pos_data = train[train["labels"] == "PO"]["texts"]
neg_data = train[train["labels"] == "NG"]["texts"]
neu_data = train[train["labels"] == "NE"]["texts"]

# 2) Criar um vetorizer para unigrams
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode")

# 3) Fazer o fit para todo o corpus
vectorizer.fit(train["texts"])

# 4) Pegar todo o vocabulário do corpus
vocab = list(vectorizer.vocabulary_.keys())
vocab_indexed = {value: key for key, value in vectorizer.vocabulary_.items()}

# 5) Obter as matrizes das classes
pos_matrix = vectorizer.transform(pos_data)
neg_matrix = vectorizer.transform(neg_data)
neu_matrix = vectorizer.transform(neu_data)

# 6) Obter o vetor de soma das frquencias das palavras nas classes
pos_sum = pos_matrix.sum(axis=0)
neg_sum = neg_matrix.sum(axis=0)
neu_sum = neu_matrix.sum(axis=0)

# 7) Construir um dicionario de frequencias para cada classe e calcular o PD de cada uma
# função que calcula o PD
pd = lambda c1, c2, c3: max([(c1 - c2 - c3)/(c1 + c2 + c3), (c2 - c1 - c3)/(c1 + c2 + c3), (c3 - c2 - c1)/(c1 + c2 + c3)])
freq_dict = [ {"PO": pos_sum[0,index], "NG": neg_sum[0,index], "NE": neu_sum[0,index], "PD": pd(pos_sum[0,index], neg_sum[0,index], neu_sum[0,index])} for index in vocab_indexed.keys()]
freq_df = DataFrame(data = freq_dict, index= vocab_indexed.values())
freq_df = freq_df[["PO", "NG", "NE", "PD"]] # Reordenando as colunas do data frame

# 8) Selecionar as melhores features
threshold = 1
selected = freq_df[freq_df["PD"] >= threshold].index.values
selected = list(selected)
print("Dimensionalidade: ", len(selected))

# 9) Avaliar o desempenho dos classificadores utilizando as features selecionadas
unigram_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words("portuguese"), strip_accents= "unicode", vocabulary= selected)
evaluate(all_data, unigram_vectorizer, 10)

Dimensionalidade:  17884
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.659379535559
Desvio padrão:  0.0346981096333

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.721770682148
Desvio padrão:  0.0315928363612

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.541364296081
Desvio padrão:  0.0429905900412


In [3]:
selected

['aabandonoo pre',
 'abafaram dentro',
 'abaixo medias',
 'abalo conviccoes',
 'abandonada diante',
 'abandonando paleto',
 'abandonar desacelerar',
 'abandono meta',
 'abandono metas',
 'abandono sete',
 'abandonou estrategia',
 'abastecido petrobras',
 'abastecimento empresa',
 'abastecimento petrobras',
 'abastecimento pp',
 'abastecimento sao',
 'aberta ex',
 'aberta nome',
 'aberto acredita',
 'aberto aqui',
 'aberto calcadao',
 'aberto revista',
 'abertura campanha',
 'abertura startups',
 'abordados relatorio',
 'abordagem distorce',
 'abordagem fere',
 'abordou corrupcao',
 'abordou programas',
 'aborto criminalizacao',
 'aborto descriminalizacao',
 'abra mente',
 'abracada marina',
 'abracaram presidente',
 'abre brecha',
 'abre pequeno',
 'abrem fecham',
 'abreu lima',
 'abril deste',
 'abrir mao',
 'abrir ultimos',
 'absoluta certeza',
 'absoluta incapacidade',
 'absoluta solucao',
 'absolutamente contra',
 'absolutamente divulgados',
 'absolutamente estranha',
 'absolutamen

In [10]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest,chi2, mutual_info_classif
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords

features = Pipeline([
                    ("lexicon_vector", CountVectorizer(strip_accents= "unicode", ngram_range=(1,2), stop_words=stopwords.words("portuguese"), vocabulary= selected)),
                    ("feature_selection", TruncatedSVD(n_components = 6000))
                    ])

run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds = 10, shuffle= True)


Cross Validation:
Accuracia media:  0.721806966618
Desvio padrão:  0.0320216565807
